In [1]:
import pandas as pd
from collections import Counter
from pathlib import Path

# ==========================
# Cấu hình
# ==========================
TOP_N = 50  # ⬅️ chỉnh ở đây nếu muốn đổi số lượng mã lấy
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
PROC_DIR = DATA_ROOT / "mimiciv_subset"
PROC_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# HÀM: LẤY & LƯU TOP MÃ PHỔ BIẾN
# ==========================
def export_top_codes(HOSP_DIR, PROC_DIR, TOP_N=50):
    # --- 1️⃣ ICD (diagnoses_icd) ---
    diag_path = HOSP_DIR / "diagnoses_icd.csv.gz"
    print("📖 Đang xử lý ICD...")
    diag_cnt = Counter()
    for chunk in pd.read_csv(diag_path, usecols=["hadm_id", "icd_code"], compression="gzip", chunksize=200_000):
        chunk = chunk.dropna(subset=["hadm_id", "icd_code"])
        diag_cnt.update(chunk["icd_code"].astype(str))
    diag_df = pd.DataFrame(diag_cnt.items(), columns=["icd_code", "count"]).sort_values("count", ascending=False)
    diag_top = diag_df.head(TOP_N)
    out_icd = PROC_DIR / "icd.csv"
    diag_top.to_csv(out_icd, index=False)
    print(f"✅ Lưu {TOP_N} mã ICD → {out_icd} ({len(diag_df):,} mã, tổng {diag_df['count'].sum():,} lượt)")

    # --- 2️⃣ Procedures (procedures_icd) ---
    proc_path = HOSP_DIR / "procedures_icd.csv.gz"
    print("🛠️  Đang xử lý thủ thuật...")
    proc_cnt = Counter()
    for chunk in pd.read_csv(proc_path, usecols=["hadm_id", "icd_code"], compression="gzip", chunksize=200_000):
        chunk = chunk.dropna(subset=["hadm_id", "icd_code"])
        proc_cnt.update(chunk["icd_code"].astype(str))
    proc_df = pd.DataFrame(proc_cnt.items(), columns=["proc_code", "count"]).sort_values("count", ascending=False)
    proc_top = proc_df.head(TOP_N)
    out_proc = PROC_DIR / "procedures.csv"
    proc_top.to_csv(out_proc, index=False)
    print(f"✅ Lưu {TOP_N} thủ thuật → {out_proc} ({len(proc_df):,} mã, tổng {proc_df['count'].sum():,} lượt)")

    # --- 3️⃣ Labs (labevents + d_labitems) ---
    lab_path = HOSP_DIR / "labevents.csv.gz"
    dlab_path = HOSP_DIR / "d_labitems.csv.gz"
    print("🧪 Đang xử lý xét nghiệm...")
    lab_cnt = Counter()
    for chunk in pd.read_csv(lab_path, usecols=["hadm_id", "itemid"], compression="gzip", chunksize=200_000):
        chunk = chunk.dropna(subset=["hadm_id", "itemid"])
        lab_cnt.update(chunk["itemid"].astype(int))
    lab_df = pd.DataFrame(lab_cnt.items(), columns=["itemid", "count"]).sort_values("count", ascending=False)

    # Gắn label từ d_labitems
    dlab = pd.read_csv(dlab_path, usecols=["itemid", "label"], compression="gzip")
    lab_df = lab_df.merge(dlab, on="itemid", how="left")
    lab_top = lab_df.head(TOP_N)
    out_lab = PROC_DIR / "labs.csv"
    lab_top.to_csv(out_lab, index=False)
    print(f"✅ Lưu {TOP_N} xét nghiệm → {out_lab} ({len(lab_df):,} mã, tổng {lab_df['count'].sum():,} lượt)")

    # --- Tổng kết ---
    print("\n=== TỔNG KẾT ===")
    print(f"- Tổng mã ICD: {len(diag_df):,}")
    print(f"- Tổng mã thủ thuật: {len(proc_df):,}")
    print(f"- Tổng mã xét nghiệm: {len(lab_df):,}")
    print(f"- Tổng lượt ICD ghi nhận: {diag_df['count'].sum():,}")
    print(f"- Tổng lượt thủ thuật ghi nhận: {proc_df['count'].sum():,}")
    print(f"- Tổng lượt xét nghiệm ghi nhận: {lab_df['count'].sum():,}")
    print(f"📂 Các file đã lưu trong {PROC_DIR.resolve()}")

# ==========================
# GỌI HÀM
# ==========================
export_top_codes(HOSP_DIR, PROC_DIR, TOP_N)


📖 Đang xử lý ICD...
✅ Lưu 50 mã ICD → ../data/mimiciv_subset/icd.csv (28,562 mã, tổng 6,364,488 lượt)
🛠️  Đang xử lý thủ thuật...
✅ Lưu 50 thủ thuật → ../data/mimiciv_subset/procedures.csv (14,911 mã, tổng 859,655 lượt)
🧪 Đang xử lý xét nghiệm...
✅ Lưu 50 xét nghiệm → ../data/mimiciv_subset/labs.csv (896 mã, tổng 84,605,867 lượt)

=== TỔNG KẾT ===
- Tổng mã ICD: 28,562
- Tổng mã thủ thuật: 14,911
- Tổng mã xét nghiệm: 896
- Tổng lượt ICD ghi nhận: 6,364,488
- Tổng lượt thủ thuật ghi nhận: 859,655
- Tổng lượt xét nghiệm ghi nhận: 84,605,867
📂 Các file đã lưu trong /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv_subset


In [2]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình đường dẫn
# ==========================
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

PROC_DIR = SUBSET_DIR

# ==========================
# 1️⃣ Đọc top 50 mã phổ biến
# ==========================
top_icd = pd.read_csv(PROC_DIR / "icd.csv")["icd_code"].astype(str).tolist()
top_proc = pd.read_csv(PROC_DIR / "procedures.csv")["proc_code"].astype(str).tolist()
top_lab = pd.read_csv(PROC_DIR / "labs.csv")["itemid"].astype(int).tolist()

print(f"🔍 Đọc vocab top50: ICD={len(top_icd)}, PROC={len(top_proc)}, LAB={len(top_lab)}")

# ==========================
# 2️⃣ Hàm tiện ích: lấy danh sách hadm_id theo mã
# ==========================
def get_hadm_with_codes(path, col_code, top_list, dtype="str"):
    hadm_set = set()
    for chunk in pd.read_csv(path, usecols=["hadm_id", col_code], chunksize=200_000):
        chunk = chunk.dropna(subset=["hadm_id", col_code])
        if dtype == "str":
            chunk[col_code] = chunk[col_code].astype(str)
        else:
            chunk[col_code] = chunk[col_code].astype(int)
        matched = chunk.loc[chunk[col_code].isin(top_list), "hadm_id"].astype(int)
        hadm_set.update(matched.tolist())
    return hadm_set

# ==========================
# 3️⃣ Lọc theo từng loại
# ==========================
print("📖 Đang lọc hadm theo top ICD...")
hadm_icd = get_hadm_with_codes(HOSP_DIR / "diagnoses_icd.csv.gz", "icd_code", top_icd, "str")
print(f"→ {len(hadm_icd):,} lượt nhập viện có ICD trong top 50")

print("🛠️  Đang lọc hadm theo top thủ thuật...")
hadm_proc = get_hadm_with_codes(HOSP_DIR / "procedures_icd.csv.gz", "icd_code", top_proc, "str")
print(f"→ {len(hadm_proc):,} lượt nhập viện có thủ thuật trong top 50")

print("🧪 Đang lọc hadm theo top xét nghiệm...")
hadm_lab = get_hadm_with_codes(HOSP_DIR / "labevents.csv.gz", "itemid", top_lab, "int")
print(f"→ {len(hadm_lab):,} lượt nhập viện có xét nghiệm trong top 50")

print("🩺 Đang lọc hadm có ghi chú xuất viện...")
hadm_note = set()
for chunk in pd.read_csv(NOTE_DIR / "discharge.csv.gz", usecols=["hadm_id", "text"], chunksize=200_000):
    chunk = chunk.dropna(subset=["hadm_id", "text"])
    hadm_note.update(chunk["hadm_id"].astype(int).tolist())
print(f"→ {len(hadm_note):,} lượt nhập viện có note xuất viện")

# ==========================
# 4️⃣ Giao 4 tập
# ==========================
hadm_all = hadm_icd & hadm_proc & hadm_lab & hadm_note
print(f"\n✅ Tổng số lượt nhập viện có đủ 4 yếu tố (ICD + PROC + LAB + NOTE): {len(hadm_all):,}")

# ==========================
# 5️⃣ Lọc bảng admissions và lưu ra CSV thường
# ==========================
adm_path = HOSP_DIR / "admissions.csv.gz"
print("📘 Đang lọc bảng admissions...")

admissions = pd.read_csv(
    adm_path,
    usecols=["hadm_id", "subject_id", "admittime", "dischtime", "deathtime"],
    compression="gzip",
    parse_dates=["admittime", "dischtime", "deathtime"],
)

admissions = admissions[admissions["hadm_id"].isin(hadm_all)].copy()
admissions = admissions.sort_values("admittime")

out_path = SUBSET_DIR / "admissions.csv"
admissions.to_csv(out_path, index=False)  # 💾 không nén gzip

print(f"📂 Đã lưu subset admissions với {len(admissions):,} lượt nhập viện → {out_path.resolve()}")
print("Bao gồm các cột: hadm_id, subject_id, admittime, dischtime, deathtime")


🔍 Đọc vocab top50: ICD=50, PROC=50, LAB=50
📖 Đang lọc hadm theo top ICD...
→ 454,431 lượt nhập viện có ICD trong top 50
🛠️  Đang lọc hadm theo top thủ thuật...
→ 145,381 lượt nhập viện có thủ thuật trong top 50
🧪 Đang lọc hadm theo top xét nghiệm...
→ 441,727 lượt nhập viện có xét nghiệm trong top 50
🩺 Đang lọc hadm có ghi chú xuất viện...
→ 331,793 lượt nhập viện có note xuất viện

✅ Tổng số lượt nhập viện có đủ 4 yếu tố (ICD + PROC + LAB + NOTE): 87,951
📘 Đang lọc bảng admissions...
📂 Đã lưu subset admissions với 87,951 lượt nhập viện → /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv_subset/admissions.csv
Bao gồm các cột: hadm_id, subject_id, admittime, dischtime, deathtime


In [3]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình
# ==========================
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# 1️⃣ Đọc danh sách bệnh nhân trong subset admissions
# ==========================
adm_subset_path = SUBSET_DIR / "admissions.csv"
if not adm_subset_path.exists():
    adm_subset_path = SUBSET_DIR / "admissions.csv.gz"

admissions = pd.read_csv(adm_subset_path, usecols=["hadm_id", "subject_id"])
subject_ids = set(admissions["subject_id"].dropna().astype(int).tolist())

print(f"🔍 Tổng số bệnh nhân cần giữ: {len(subject_ids):,}")

# ==========================
# 2️⃣ Đọc và lọc bảng patients
# ==========================
patients_path = HOSP_DIR / "patients.csv.gz"
usecols = ["subject_id", "gender", "anchor_age", "anchor_year"]

patients = pd.read_csv(patients_path, usecols=usecols, compression="gzip")
patients = patients[patients["subject_id"].isin(subject_ids)].copy()

print(f"✅ Đã lọc {len(patients):,} bệnh nhân trong tập admissions subset")

# ==========================
# 3️⃣ Chuẩn hoá giới tính và lưu CSV (không nén)
# ==========================
patients["gender"] = patients["gender"].astype(str).str.upper().str[0]  # chuẩn hóa 'M', 'F', 'U'

out_path = SUBSET_DIR / "patients.csv"
patients.to_csv(out_path, index=False)

print(f"📂 Đã lưu file rút gọn → {out_path.resolve()}")
print("Bao gồm các cột: subject_id, gender, anchor_age, anchor_year")


🔍 Tổng số bệnh nhân cần giữ: 52,610
✅ Đã lọc 52,610 bệnh nhân trong tập admissions subset
📂 Đã lưu file rút gọn → /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv_subset/patients.csv
Bao gồm các cột: subject_id, gender, anchor_age, anchor_year


In [6]:
import pandas as pd
import re
from pathlib import Path

# ==========================
# Cấu hình đường dẫn
# ==========================
DATA_ROOT = Path("../data")
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

admissions_path = SUBSET_DIR / "admissions.csv"
note_path = NOTE_DIR / "discharge.csv.gz"
out_path = SUBSET_DIR / "discharge.csv.gz"

# ==========================
# Đọc danh sách hadm_id trong subset
# ==========================
admissions = pd.read_csv(admissions_path, usecols=["hadm_id"])
hadm_ids = set(admissions["hadm_id"].astype(int))
print(f"📋 Tổng số lượt nhập viện cần lấy: {len(hadm_ids):,}")

# ==========================
# Hàm làm sạch ghi chú
# ==========================
import re

def clean_discharge_text(text: str) -> str:
    """
    Extract specific sections from a MIMIC-IV discharge note: Service, Chief Complaint,
    History of Present Illness, Past Medical History, Family History, and Physical Exam
    (admission only). Maintains compatibility with the original function's input/output format
    and cleaning logic.

    Args:
        text (str): Raw discharge note text.

    Returns:
        str: Cleaned text containing only the specified sections, with excessive whitespace
             and lab result lines removed.
    """
    if not isinstance(text, str) or len(text.strip()) == 0:
        return ""

    # Define sections to keep
    sections_to_keep = [
        r"Service:\s*",
        r"Chief Complaint:\s*",
        r"History of Present Illness:\s*",
        r"Past Medical History:\s*",
        r"Family History:\s*",
        r"Physical Exam:\s*"
    ]

    # Combine section headers into a regex pattern
    section_pattern = r"^(?:" + "|".join(sections_to_keep) + r")"
    section_headers = [re.escape(s).rstrip(r"\s*") for s in sections_to_keep]

    # Split text into lines
    lines = text.splitlines()
    result = []
    capture = False
    current_section = None
    physical_exam_count = 0  # Track to capture only the first Physical Exam

    for line in lines:
        # Check if the line starts with a desired section header
        if re.match(section_pattern, line, re.MULTILINE | re.IGNORECASE):
            section_name = next((h for h in section_headers if line.lower().startswith(h.lower())), None)
            if section_name:
                # Only capture the first Physical Exam (admission)
                if section_name == r"Physical Exam:":
                    physical_exam_count += 1
                    if physical_exam_count > 1:  # Skip subsequent Physical Exam sections
                        capture = False
                        continue
                current_section = section_name
                capture = True
                result.append(line)
                continue
        # Capture lines for the current section
        if capture:
            # Stop capturing if we hit another section header or "Discharge"
            if re.match(r"^[A-Z][a-zA-Z\s]*:\s*$", line, re.MULTILINE) or "Discharge" in line:
                capture = False
                continue
            result.append(line)

    # Join lines and clean up
    text = "\n".join(result).strip()
    
    # Remove lab result-like lines (e.g., "Na-136", "WBC-5.0") to match original function
    text = re.sub(r"\b[A-Z]{2,}[A-Za-z0-9/%\-]*-?\d+(\.\d+)?\*?\b", "", text)
    
    # Collapse excessive newlines to match original function
    text = re.sub(r"\n{3,}", "\n\n", text).strip()

    return text

# ==========================
# Đọc file discharge, lọc theo hadm_id, làm sạch, lưu lại .gz
# ==========================
out_chunks = []
for chunk in pd.read_csv(note_path, usecols=["hadm_id", "charttime", "text"], compression="gzip", chunksize=20_000):
    chunk = chunk.dropna(subset=["hadm_id", "text"])
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk = chunk[chunk["hadm_id"].isin(hadm_ids)]

    if not chunk.empty:
        chunk["text"] = chunk["text"].astype(str).apply(clean_discharge_text)
        out_chunks.append(chunk)

df = pd.concat(out_chunks, ignore_index=True)
df.to_csv(out_path, index=False, compression="gzip")

print(f"✅ Đã lưu {len(df):,} ghi chú rút gọn → {out_path}")
print(f"💾 Dung lượng sau nén: nhỏ gọn & sẵn sàng cho training hoặc phân tích NLP")


📋 Tổng số lượt nhập viện cần lấy: 87,951
✅ Đã lưu 87,951 ghi chú rút gọn → ../data/mimiciv_subset/discharge.csv.gz
💾 Dung lượng sau nén: nhỏ gọn & sẵn sàng cho training hoặc phân tích NLP


In [5]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình đường dẫn
# ==========================
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# 1️⃣ Hàm tiện ích
# ==========================
def export_filtered_dict(top_file, dict_file, out_name, code_col="icd_code"):
    print(f"\n📘 Xử lý {top_file.name} → {out_name}")
    
    # Đọc danh sách top
    top_df = pd.read_csv(top_file)
    top_codes = top_df[top_df.columns[0]].astype(str).tolist()
    
    # Đọc dictionary gốc
    dict_df = pd.read_csv(dict_file, compression="gzip")
    dict_df[code_col] = dict_df[code_col].astype(str)
    
    # Lọc theo top
    filtered = dict_df[dict_df[code_col].isin(top_codes)].copy()
    
    # Merge thêm cột count để tiện tra cứu
    filtered = filtered.merge(top_df, left_on=code_col, right_on=top_df.columns[0], how="left")
    filtered = filtered.sort_values("count", ascending=False)
    
    # Lưu file với tên gốc
    out_path = SUBSET_DIR / out_name
    filtered.to_csv(out_path, index=False)
    
    print(f"✅ Lưu {len(filtered):,} dòng → {out_path}")
    return filtered

# ==========================
# 2️⃣ ICD Diagnoses
# ==========================
diag_filtered = export_filtered_dict(
    top_file = SUBSET_DIR / "icd.csv",
    dict_file = HOSP_DIR / "d_icd_diagnoses.csv.gz",
    out_name  = "d_icd_diagnoses.csv",
    code_col  = "icd_code"
)

# ==========================
# 3️⃣ ICD Procedures
# ==========================
proc_filtered = export_filtered_dict(
    top_file = SUBSET_DIR / "procedures.csv",
    dict_file = HOSP_DIR / "d_icd_procedures.csv.gz",
    out_name  = "d_icd_procedures.csv",
    code_col  = "icd_code"
)

# ==========================
# 4️⃣ Tổng kết
# ==========================
print("\n=== TỔNG KẾT ===")
print(f"- d_icd_diagnoses.csv: {len(diag_filtered):,} dòng")
print(f"- d_icd_procedures.csv: {len(proc_filtered):,} dòng")
print(f"📂 Đã lưu vào: {SUBSET_DIR.resolve()}")



📘 Xử lý icd.csv → d_icd_diagnoses.csv
✅ Lưu 50 dòng → ../data/mimiciv_subset/d_icd_diagnoses.csv

📘 Xử lý procedures.csv → d_icd_procedures.csv
✅ Lưu 50 dòng → ../data/mimiciv_subset/d_icd_procedures.csv

=== TỔNG KẾT ===
- d_icd_diagnoses.csv: 50 dòng
- d_icd_procedures.csv: 50 dòng
📂 Đã lưu vào: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv_subset


In [3]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình đường dẫn
# ==========================
DATA_ROOT = Path("../data")
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# 1️⃣ Đọc danh sách hadm_id từ admissions subset
# ==========================
adm_path = SUBSET_DIR / "admissions.csv"
admissions = pd.read_csv(adm_path, usecols=["hadm_id"])
hadm_set = set(admissions["hadm_id"].astype(int))
print(f"📋 Đọc {len(hadm_set):,} lượt nhập viện trong subset admissions.")

# ==========================
# 2️⃣ Đọc discharge note và lọc theo hadm_id
# ==========================
note_path = NOTE_DIR / "discharge.csv.gz"
out_path = SUBSET_DIR / "discharge_full.csv.gz"

rows = []
for chunk in pd.read_csv(
    note_path, 
    usecols=["hadm_id", "charttime", "text"], 
    compression="gzip",
    chunksize=100_000
):
    chunk = chunk.dropna(subset=["hadm_id", "text"]).copy()
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    matched = chunk[chunk["hadm_id"].isin(hadm_set)]
    if not matched.empty:
        rows.append(matched)

if rows:
    df_out = pd.concat(rows, ignore_index=True)
else:
    df_out = pd.DataFrame(columns=["hadm_id", "charttime", "text"])

# ==========================
# 3️⃣ Lưu kết quả ra file nén gzip
# ==========================
df_out.to_csv(out_path, index=False, compression="gzip")
print(f"✅ Đã lưu {len(df_out):,} ghi chú xuất viện vào → {out_path}")


📋 Đọc 87,951 lượt nhập viện trong subset admissions.
✅ Đã lưu 87,951 ghi chú xuất viện vào → ../data/mimiciv_subset/discharge_full.csv.gz


In [1]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình
# ==========================
DATA_ROOT = Path("../data")
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# Đọc danh sách lượt nhập viện trong subset
# ==========================
adm_subset_path = SUBSET_DIR / "admissions.csv"
admissions = pd.read_csv(adm_subset_path, usecols=["hadm_id"])
hadm_set = set(admissions["hadm_id"].astype(int))

# ==========================
# Lấy 1 ghi chú xuất viện đầy đủ (đầu tiên trong subset)
# ==========================
note_path = NOTE_DIR / "discharge.csv.gz"
for chunk in pd.read_csv(note_path, usecols=["hadm_id", "charttime", "text"], compression="gzip", chunksize=50_000):
    chunk = chunk.dropna(subset=["hadm_id", "text"])
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    match = chunk[chunk["hadm_id"].isin(hadm_set)]
    if not match.empty:
        sample_df = match.iloc[[0]]
        break
else:
    sample_df = pd.DataFrame(columns=["hadm_id", "charttime", "text"])

# ==========================
# Lưu ra file CSV (1 ghi chú duy nhất)
# ==========================
out_path = SUBSET_DIR / "sample_discharge.csv"
sample_df.to_csv(out_path, index=False)


In [8]:
df.sample(1).to_csv("../data/mimiciv_subset/discharge_example.csv", index=False)

In [11]:
import pandas as pd
from pathlib import Path

# ==============================
# CẤU HÌNH
# ==============================
DATA_ROOT = Path("../data")
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
HADM_ID = 27783915  # 🏥 Lần nhập viện cần kiểm tra

# ==============================
# 1️⃣ ĐỌC DANH SÁCH TOP MÃ
# ==============================
top_icd = pd.read_csv(SUBSET_DIR / "icd.csv")["icd_code"].astype(str).tolist()
top_proc = pd.read_csv(SUBSET_DIR / "procedures.csv")["proc_code"].astype(str).tolist()
top_lab = pd.read_csv(SUBSET_DIR / "labs.csv")["itemid"].astype(int).tolist()

# ==============================
# 2️⃣ ĐỌC CÁC FILE CỦA BỆNH NHÂN
# ==============================
diag = pd.read_csv(SUBSET_DIR / f"diagnoses_{HADM_ID}.csv")
proc = pd.read_csv(SUBSET_DIR / f"procedures_{HADM_ID}.csv")
labs = pd.read_csv(SUBSET_DIR / f"labs_{HADM_ID}.csv")

# ==============================
# 3️⃣ ĐẾM SỐ LƯỢNG NẰM TRONG TOP
# ==============================
diag_in_top = diag[diag["icd_code"].astype(str).isin(top_icd)]
proc_in_top = proc[proc["icd_code"].astype(str).isin(top_proc)]
labs_in_top = labs[labs["itemid"].astype(int).isin(top_lab)]

summary = {
    "hadm_id": HADM_ID,
    "total_diagnoses": len(diag),
    "total_procedures": len(proc),
    "total_labs": len(labs),
    "diagnoses_in_top": len(diag_in_top),
    "procedures_in_top": len(proc_in_top),
    "labs_in_top": len(labs_in_top),
}

summary_df = pd.DataFrame([summary])
summary_path = SUBSET_DIR / f"summary_{HADM_ID}.csv"
summary_df.to_csv(summary_path, index=False)

# ==============================
# 4️⃣ IN KẾT QUẢ
# ==============================
print("📊 TỔNG KẾT CHO LẦN NHẬP VIỆN", HADM_ID)
print(f"- Tổng số chẩn đoán: {summary['total_diagnoses']:,}  |  Nằm trong top: {summary['diagnoses_in_top']:,}")
print(f"- Tổng số thủ thuật: {summary['total_procedures']:,}  |  Nằm trong top: {summary['procedures_in_top']:,}")
print(f"- Tổng số xét nghiệm: {summary['total_labs']:,}  |  Nằm trong top: {summary['labs_in_top']:,}")
print(f"📂 Đã lưu thống kê → {summary_path}")


📊 TỔNG KẾT CHO LẦN NHẬP VIỆN 27783915
- Tổng số chẩn đoán: 25  |  Nằm trong top: 8
- Tổng số thủ thuật: 7  |  Nằm trong top: 5
- Tổng số xét nghiệm: 230  |  Nằm trong top: 189
📂 Đã lưu thống kê → ../data/mimiciv_subset/summary_27783915.csv


In [6]:
import pandas as pd
from pathlib import Path

# Đường dẫn
DATA_ROOT = Path("../data")
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
file_path = SUBSET_DIR / "discharge_full.csv.gz"

# Đọc 1 dòng bất kỳ (ở đây là dòng đầu tiên)
sample = pd.read_csv(file_path, nrows=1)
print(sample.iloc[0]["text"])


 
Name:  ___                     Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___
 
Chief Complaint:
Worsening ABD distension and pain 
 
Major Surgical or Invasive Procedure:
Paracentesis

 
History of Present Illness:
___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, 
bioplar, PTSD, presented from OSH ED with worsening abd 
distension over past week.  
Pt reports self-discontinuing lasix and spirnolactone ___ weeks 
ago, because she feels like "they don't do anything" and that 
she "doesn't want to put more chemicals in her." She does not 
follow Na-restricted diets. In the past week, she notes that she 
has been having worsening abd distension and discomfort. She 
denies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, 
dysuria. She had food poisoning a week ago from eating stale 
cake (n/v 20 min after fo

In [9]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình
# ==========================
SUBSET_DIR = Path("../data/mimiciv_subset")
discharge_path = SUBSET_DIR / "discharge_full.csv.gz"

# ==========================
# Đọc file và tính số từ
# ==========================
df = pd.read_csv(discharge_path, usecols=["text"])

# Loại bỏ NaN & tính số từ trong mỗi ghi chú
df["word_count"] = df["text"].fillna("").apply(lambda x: len(str(x).split()))

avg_words = df["word_count"].mean()
max_words = df["word_count"].max()
min_words = df["word_count"].min()

print(f"🩺 Số ghi chú: {len(df):,}")
print(f"📊 Trung bình: {avg_words:,.0f} từ/note")
print(f"🔹 Ngắn nhất: {min_words:,} từ")
print(f"🔸 Dài nhất: {max_words:,} từ")


🩺 Số ghi chú: 87,951
📊 Trung bình: 1,866 từ/note
🔹 Ngắn nhất: 44 từ
🔸 Dài nhất: 9,026 từ


In [6]:
import re
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình đường dẫn
# ==========================
DATA_ROOT  = Path("../data")
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
NOTE_DIR   = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"  # chỉ dùng khi fallback
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

# Nguồn note ưu tiên: file full đã gom về subset
note_path = SUBSET_DIR / "discharge_full.csv.gz"
if not note_path.exists():
    # fallback nếu chưa có file subset
    note_path = NOTE_DIR / "discharge.csv.gz"

adm_path  = SUBSET_DIR / "admissions.csv"   # danh sách hadm trong subset
out_path  = SUBSET_DIR / "discharge.csv.gz" # OUTPUT

CHUNKSIZE = 200_000

# ==========================
# 1) Tập HADM cần lấy
# ==========================
admissions = pd.read_csv(adm_path, usecols=["hadm_id"])
HADM_SET = set(admissions["hadm_id"].astype(int))

# ==========================
# 2) Regex & hàm trích xuất 6 mục
# ==========================
# Headings cần giữ (key: label chuẩn, value: pattern để match tiêu đề)
HEADINGS = {
    "Chief Complaint": r"Chief Complaint",
    "Major Surgical or Invasive Procedure": r"Major Surgical\s+or\s+Invasive Procedure",
    "History of Present Illness": r"History of Present Illness",
    "Past Medical History": r"Past Medical History",
    # Physical Exam có thể xuất hiện ở dạng "Physical Exam" hoặc "PHYSICAL EXAMINATION"
    "Physical Exam": r"(?:Physical Exam|PHYSICAL EXAMINATION)",
    "Pertinent Results": r"Pertinent Results",
}

# Mẫu để xác định ranh giới các heading nói chung (bắt đầu dòng, cụm chữ cái + có thể khoảng trắng rồi dấu :)
GEN_HEADING = re.compile(r"^[A-Z][A-Za-z/ ()'#\.\-]*:\s*$", flags=re.M)

def extract_selected_sections(text: str) -> str:
    """Trích 6 mục và gộp lại thành 1 text."""
    if not isinstance(text, str) or not text.strip():
        return ""
    # Chuẩn hoá xuống dạng có dòng trống phân cách nhẹ
    s = re.sub(r"\r\n?", "\n", text)

    # Tìm tất cả heading vị trí
    # Chiến lược: tìm vị trí của từng heading cụ thể cần giữ; cắt đến heading kế tiếp bất kỳ (GEN_HEADING)
    pieces = []
    for label, pat in HEADINGS.items():
        m = re.search(rf"(?mi)^\s*{pat}\s*:\s*$", s)
        if not m:
            continue
        start = m.end()  # bắt đầu nội dung sau tiêu đề này

        # Tìm heading kế tiếp bất kỳ
        next_m = GEN_HEADING.search(s, pos=start)
        end = next_m.start() if next_m else len(s)

        body = s[start:end].strip()
        if body:
            pieces.append(f"{label}:\n{body}")

    return "\n\n".join(pieces).strip()

# ==========================
# 3) Đọc note theo chunk, lọc HADM, trích & gộp text
# ==========================
rows = []
usecols = ["hadm_id", "charttime", "text"]

for chunk in pd.read_csv(note_path, usecols=usecols, compression="gzip", chunksize=CHUNKSIZE):
    if chunk.empty:
        continue
    chunk = chunk.dropna(subset=["hadm_id", "text"]).copy()
    if chunk.empty:
        continue

    chunk["hadm_id"]  = chunk["hadm_id"].astype(int)
    chunk = chunk[chunk["hadm_id"].isin(HADM_SET)]
    if chunk.empty:
        continue

    # Giữ record mới nhất theo hadm_id nếu có trùng (sort desc charttime, drop_duplicates keep first)
    if "charttime" in chunk.columns:
        # parse vừa đủ: chỉ với chunk này để đảm bảo đúng sort
        chunk["charttime"] = pd.to_datetime(chunk["charttime"], errors="coerce")
        chunk = (
            chunk.sort_values(["hadm_id", "charttime"], ascending=[True, False])
                 .drop_duplicates(subset=["hadm_id"], keep="first")
        )

    # Trích 6 mục và gộp text
    chunk["text"] = chunk["text"].astype(str).map(extract_selected_sections)

    # Chỉ giữ các bản có text sau khi trích (không rỗng)
    chunk = chunk[chunk["text"].str.len() > 0]

    if not chunk.empty:
        rows.append(chunk[["hadm_id", "charttime", "text"]])

# ==========================
# 4) Gộp & lưu output
# ==========================
if rows:
    out_df = pd.concat(rows, ignore_index=True)
    # Nếu có trùng hadm giữa các chunk (hiếm), giữ bản mới nhất
    if "charttime" in out_df.columns:
        out_df = (
            out_df.sort_values(["hadm_id", "charttime"], ascending=[True, False])
                  .drop_duplicates(subset=["hadm_id"], keep="first")
                  .sort_values("hadm_id")
        )
else:
    out_df = pd.DataFrame(columns=["hadm_id", "charttime", "text"])

out_df.to_csv(out_path, index=False, compression="gzip")
print(f"✅ Đã lưu {len(out_df):,} note đã gộp 6 mục → {out_path}")


✅ Đã lưu 87,544 note đã gộp 6 mục → ../data/mimiciv_subset/discharge.csv.gz


In [13]:
import pandas as pd
import numpy as np
from pathlib import Path

SUBSET_DIR = Path("../data/mimiciv_subset")
path = SUBSET_DIR / "discharge.csv.gz"

rng = np.random.default_rng(1)  # tái lập
sample = None
seen = 0

usecols = ["hadm_id", "charttime", "text"]

for chunk in pd.read_csv(path, compression="gzip", usecols=usecols, chunksize=100_000):
    # loại NaN để tránh lỗi in ấn
    chunk["text"] = chunk["text"].fillna("")
    for _, row in chunk.iterrows():
        seen += 1
        if rng.random() < 1.0 / seen:  # reservoir sampling
            sample = row

if sample is None:
    print("File rỗng?")
else:
    print("hadm_id:", sample["hadm_id"])
    print("charttime:", sample["charttime"])
    print("\n--- TEXT ---\n")
    # in nguyên văn toàn bộ ghi chú
    print(str(sample["text"]))


hadm_id: 24151113
charttime: 2153-02-07

--- TEXT ---

Chief Complaint:
Febrile neutropenia, bacteremia

Major Surgical or Invasive Procedure:
None

History of Present Illness:
Ms. ___ is a ___ year-old female with an unclear 
neurologic condition (currently being evaluated at ___, 
hepatitis C s/p 12 weeks of Harvoni now with a negative VL, and 
MDS diagnosed in ___, now transfusion-dependent, referred to 
___ for consideration of reduced intensity allogeneic stem 
cell transplantation, presented to OSH with febrile neutropenia 
and coag-neg staph bacteremia, and transferred here for further 
care. 

On ___, she developed fevers, chills, and weakness 
without any localizing symptoms (no cough, abdominal pain, 
dysuria, headaches, chest pain, rashes). She developed a fever 
to 102.1, called her physician, who encouraged her to present 
emergently to the ED. In OSH ED, blood cultures were drawn, and 
there was concern for UTI so zosyn was started. Blood cx grew 
GPCs, so zosyn was switc

In [1]:
import pandas as pd
from pathlib import Path

SUBSET_DIR = Path("../data/mimiciv_subset")
path = SUBSET_DIR / "discharge.csv.gz"
pd.read_csv(path, compression="gzip", nrows=1)


,hadm_id,charttime,chief_complaint,major_procedures,hpi,pmh,physical_exam,pertinent_results
0,20000147,2121-09-03,chest pain \n \nMajor Surgical or Invasive Pro...,___ Cardiac catheterization \n___ Emergent cor...,___ year old male with a history of\ndiabetes ...,Hypercholesterolemia \nChronic back pain \nHyp...,Pulse:86 Resp:27 O2 sat: 99/RA\nB/P: 128/79\n...,___ 04:30PM BLOOD WBC-6.9 RBC-4.28* Hgb-13.1* ...


In [10]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình
# ==========================
SUBSET_DIR = Path("../data/mimiciv_subset")
discharge_path = SUBSET_DIR / "discharge.csv.gz"

# ==========================
# Đọc file và tính số từ
# ==========================
df = pd.read_csv(discharge_path, usecols=["hadm_id", "text"])

# Loại bỏ NaN & tính số từ trong mỗi ghi chú
df["word_count"] = df["text"].fillna("").apply(lambda x: len(str(x).split()))

avg_words = df["word_count"].mean()
max_words = df["word_count"].max()
min_words = df["word_count"].min()

# Ghi chú < 512 từ
threshold = 512
short_notes = df[df["word_count"] < threshold]

print(f"🩺 Số ghi chú: {len(df):,}")
print(f"📊 Trung bình: {avg_words:,.0f} từ/note")
print(f"🔹 Ngắn nhất: {min_words:,} từ")
print(f"🔸 Dài nhất: {max_words:,} từ")
print(f"✅ Số ghi chú < {threshold} từ: {len(short_notes):,} "
      f"(~{len(short_notes)/len(df):.1%})")

# (Tuỳ chọn) Lưu danh sách các HADM có note ngắn
# short_notes[["hadm_id", "word_count"]].to_csv(SUBSET_DIR / "discharge_short_under_512.csv", index=False)


🩺 Số ghi chú: 87,544
📊 Trung bình: 482 từ/note
🔹 Ngắn nhất: 5 từ
🔸 Dài nhất: 4,267 từ
✅ Số ghi chú < 512 từ: 56,296 (~64.3%)


In [11]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình
# ==========================
SUBSET_DIR = Path("../data/mimiciv_subset")
adm_path = SUBSET_DIR / "admissions.csv"          # admissions hiện tại (CSV)
dis_path = SUBSET_DIR / "discharge.csv.gz"        # ghi chú đã chuẩn bị

THRESH = 512  # số từ tối đa cho ghi chú

# ==========================
# 1) Lấy danh sách hadm_id có note < 512 từ
# ==========================
dis = pd.read_csv(dis_path, usecols=["hadm_id", "text"], compression="gzip")
dis["hadm_id"] = dis["hadm_id"].astype(int)
dis["word_count"] = dis["text"].fillna("").map(lambda s: len(str(s).split()))

hadm_under = set(dis.loc[dis["word_count"] < THRESH, "hadm_id"].tolist())

print(f"Số HADM có note < {THRESH} từ: {len(hadm_under):,}")

# ==========================
# 2) Đọc admissions hiện tại và lọc theo danh sách trên
# ==========================
adm = pd.read_csv(adm_path)
before = len(adm)
adm["hadm_id"] = adm["hadm_id"].astype(int)

adm_filtered = adm[adm["hadm_id"].isin(hadm_under)].copy()
after = len(adm_filtered)

# (Tuỳ chọn) Sao lưu trước khi ghi đè
# adm.to_csv(SUBSET_DIR / "admissions.backup.csv", index=False)

# ==========================
# 3) Ghi đè admissions.csv
# ==========================
adm_filtered.to_csv(adm_path, index=False)
print(f"Đã ghi đè admissions.csv — còn {after:,}/{before:,} lượt nhập viện (note < {THRESH} từ).")


Số HADM có note < 512 từ: 56,296
Đã ghi đè admissions.csv — còn 56,296/87,951 lượt nhập viện (note < 512 từ).


In [14]:
import pandas as pd
from pathlib import Path

# ===== Cấu hình đường dẫn =====
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"  # không dùng ở đây nhưng để tham chiếu
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

HADM = 24151113

# ===== 1) CHẨN ĐOÁN (Diagnoses) =====
diag_path = HOSP_DIR / "diagnoses_icd.csv.gz"
d_diag_path = HOSP_DIR / "d_icd_diagnoses.csv.gz"

# Đọc dictionary tên chẩn đoán
d_diag = pd.read_csv(d_diag_path, usecols=["icd_code", "icd_version", "long_title"], compression="gzip")
d_diag["icd_code"] = d_diag["icd_code"].astype(str)

# Lọc theo hadm_id bằng chunks để tiết kiệm RAM
diag_rows = []
for chunk in pd.read_csv(diag_path,
                         usecols=["subject_id", "hadm_id", "icd_code", "icd_version"],
                         compression="gzip",
                         chunksize=200_000):
    chunk = chunk[chunk["hadm_id"].astype(int) == HADM]
    if not chunk.empty:
        chunk["icd_code"] = chunk["icd_code"].astype(str)
        diag_rows.append(chunk)

if diag_rows:
    dx = pd.concat(diag_rows, ignore_index=True).dropna(subset=["icd_code"])
    dx = dx.merge(d_diag, on=["icd_code", "icd_version"], how="left")
    dx = dx[["subject_id","hadm_id","icd_code","icd_version","long_title"]].drop_duplicates()
else:
    dx = pd.DataFrame(columns=["subject_id","hadm_id","icd_code","icd_version","long_title"])

out_dx = SUBSET_DIR / f"{HADM}_diagnoses.csv"
dx.to_csv(out_dx, index=False)
print(f"✅ Diagnoses: {len(dx)} dòng → {out_dx}")

# ===== 2) THỦ THUẬT (Procedures) =====
proc_path = HOSP_DIR / "procedures_icd.csv.gz"
d_proc_path = HOSP_DIR / "d_icd_procedures.csv.gz"

# Đọc dictionary tên thủ thuật
d_proc = pd.read_csv(d_proc_path, usecols=["icd_code", "icd_version", "long_title"], compression="gzip")
d_proc["icd_code"] = d_proc["icd_code"].astype(str)

proc_rows = []
for chunk in pd.read_csv(proc_path,
                         usecols=["subject_id", "hadm_id", "icd_code", "icd_version"],
                         compression="gzip",
                         chunksize=200_000):
    chunk = chunk[chunk["hadm_id"].astype(int) == HADM]
    if not chunk.empty:
        chunk["icd_code"] = chunk["icd_code"].astype(str)
        proc_rows.append(chunk)

if proc_rows:
    px = pd.concat(proc_rows, ignore_index=True).dropna(subset=["icd_code"])
    px = px.merge(d_proc, on=["icd_code", "icd_version"], how="left")
    px = px[["subject_id","hadm_id","icd_code","icd_version","long_title"]].drop_duplicates()
else:
    px = pd.DataFrame(columns=["subject_id","hadm_id","icd_code","icd_version","long_title"])

out_px = SUBSET_DIR / f"{HADM}_procedures.csv"
px.to_csv(out_px, index=False)
print(f"✅ Procedures: {len(px)} dòng → {out_px}")

# ===== 3) XÉT NGHIỆM (Labs) =====
lab_path = HOSP_DIR / "labevents.csv.gz"
dlab_path = HOSP_DIR / "d_labitems.csv.gz"

# Từ điển itemid -> label
dlab = pd.read_csv(dlab_path, usecols=["itemid","label"], compression="gzip")
dlab["itemid"] = dlab["itemid"].astype(int)

lab_rows = []
for chunk in pd.read_csv(lab_path,
                         usecols=["hadm_id","itemid"],
                         compression="gzip",
                         chunksize=400_000):
    chunk = chunk.dropna(subset=["hadm_id","itemid"])
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    if HADM not in set(chunk["hadm_id"].unique()):
        continue
    sub = chunk[chunk["hadm_id"] == HADM].copy()
    if not sub.empty:
        sub["itemid"] = sub["itemid"].astype(int)
        lab_rows.append(sub[["hadm_id","itemid"]])

if lab_rows:
    labs = pd.concat(lab_rows, ignore_index=True)
    # Lấy danh sách xét nghiệm (unique) và tên
    labs = labs.drop_duplicates().merge(dlab, on="itemid", how="left")
    labs = labs[["hadm_id","itemid","label"]].drop_duplicates().sort_values(["label","itemid"])
else:
    labs = pd.DataFrame(columns=["hadm_id","itemid","label"])

out_lab = SUBSET_DIR / f"{HADM}_labs.csv"
labs.to_csv(out_lab, index=False)
print(f"✅ Labs: {len(labs)} dòng → {out_lab}")

# ===== Tóm tắt nhanh trên màn hình =====
print("\n--- SUMMARY ---")
print(f"Chẩn đoán (ICD): {len(dx)}")
print(f"Thủ thuật (ICD): {len(px)}")
print(f"Xét nghiệm (itemid): {len(labs)} (unique)")


✅ Diagnoses: 33 dòng → ../data/mimiciv_subset/24151113_diagnoses.csv
✅ Procedures: 5 dòng → ../data/mimiciv_subset/24151113_procedures.csv
✅ Labs: 127 dòng → ../data/mimiciv_subset/24151113_labs.csv

--- SUMMARY ---
Chẩn đoán (ICD): 33
Thủ thuật (ICD): 5
Xét nghiệm (itemid): 127 (unique)


In [15]:
import pandas as pd
from pathlib import Path

# ========= Cấu hình =========
DATA_ROOT   = Path("../data")
HOSP_DIR    = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
SUBSET_DIR  = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

HADM_ID = 24151113   # <-- ĐỔI hadm_id bạn muốn lấy ở đây

# ========= Tiện ích: đọc top-list (hỗ trợ 2 kiểu tên file) =========
def load_top_codes(subdir: Path, candidates: list[str], col: str, as_type="str"):
    for name in candidates:
        p = subdir / name
        if p.exists():
            df = pd.read_csv(p)
            vals = df[col]
            vals = vals.astype(int) if as_type == "int" else vals.astype(str)
            return set(vals.tolist())
    raise FileNotFoundError(f"Không tìm thấy file top codes trong {subdir} với các tên: {candidates}")

top_icd = load_top_codes(
    SUBSET_DIR,
    candidates=["top_50_icd.csv", "icd.csv"],
    col="icd_code",
    as_type="str",
)

top_proc = load_top_codes(
    SUBSET_DIR,
    candidates=["top_50_procedures.csv", "procedures.csv"],
    col="proc_code",   # lưu ý cột trong file top thủ thuật là 'proc_code'
    as_type="str",
)

top_lab = load_top_codes(
    SUBSET_DIR,
    candidates=["top_50_labs.csv", "labs.csv"],
    col="itemid",
    as_type="int",
)

print(f"Đã nạp vocab top: ICD={len(top_icd)}, PROC={len(top_proc)}, LAB={len(top_lab)}")

# ========= 1) CHẨN ĐOÁN (ICD) =========
diag_rows = []
for chunk in pd.read_csv(
    HOSP_DIR / "diagnoses_icd.csv.gz",
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version"],
    compression="gzip",
    chunksize=200_000
):
    chunk = chunk[chunk["hadm_id"] == HADM_ID]
    if chunk.empty:
        continue
    chunk["icd_code"] = chunk["icd_code"].astype(str)
    keep = chunk[chunk["icd_code"].isin(top_icd)]
    if not keep.empty:
        diag_rows.append(keep)

if diag_rows:
    dx = pd.concat(diag_rows, ignore_index=True).drop_duplicates()
    # Gắn long_title từ d_icd_diagnoses (theo phiên bản)
    dicd = pd.read_csv(
        HOSP_DIR / "d_icd_diagnoses.csv.gz",
        usecols=["icd_code", "icd_version", "long_title"],
        compression="gzip"
    )
    dx = dx.merge(dicd, on=["icd_code", "icd_version"], how="left")
else:
    dx = pd.DataFrame(columns=["subject_id","hadm_id","icd_code","icd_version","long_title"])

# ========= 2) THỦ THUẬT (Procedures) =========
proc_rows = []
for chunk in pd.read_csv(
    HOSP_DIR / "procedures_icd.csv.gz",
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version"],
    compression="gzip",
    chunksize=200_000
):
    chunk = chunk[chunk["hadm_id"] == HADM_ID]
    if chunk.empty:
        continue
    chunk["icd_code"] = chunk["icd_code"].astype(str)
    keep = chunk[chunk["icd_code"].isin(top_proc)]
    if not keep.empty:
        proc_rows.append(keep)

if proc_rows:
    pr = pd.concat(proc_rows, ignore_index=True).drop_duplicates()
    dicp = pd.read_csv(
        HOSP_DIR / "d_icd_procedures.csv.gz",
        usecols=["icd_code", "icd_version", "long_title"],
        compression="gzip"
    )
    pr = pr.merge(dicp, on=["icd_code", "icd_version"], how="left")
else:
    pr = pd.DataFrame(columns=["subject_id","hadm_id","icd_code","icd_version","long_title"])

# ========= 3) XÉT NGHIỆM (Labs) =========
lab_rows = []
for chunk in pd.read_csv(
    HOSP_DIR / "labevents.csv.gz",
    usecols=["hadm_id","itemid"],
    compression="gzip",
    chunksize=200_000
):
    chunk = chunk[chunk["hadm_id"] == HADM_ID]
    if chunk.empty:
        continue
    chunk["itemid"] = chunk["itemid"].astype(int)
    keep = chunk[chunk["itemid"].isin(top_lab)]
    if not keep.empty:
        lab_rows.append(keep)

if lab_rows:
    lb = pd.concat(lab_rows, ignore_index=True).drop_duplicates()
    dlab = pd.read_csv(
        HOSP_DIR / "d_labitems.csv.gz",
        usecols=["itemid","label"],
        compression="gzip"
    )
    lb = lb.merge(dlab, on="itemid", how="left")
else:
    lb = pd.DataFrame(columns=["hadm_id","itemid","label"])

# ========= Lưu & In nhanh =========
out_dx  = SUBSET_DIR / f"hadm_{HADM_ID}_diagnoses_top.csv"
out_pr  = SUBSET_DIR / f"hadm_{HADM_ID}_procedures_top.csv"
out_lab = SUBSET_DIR / f"hadm_{HADM_ID}_labs_top.csv"

dx.to_csv(out_dx, index=False)
pr.to_csv(out_pr, index=False)
lb.to_csv(out_lab, index=False)

print("\n=== TÓM TẮT ===")
print(f"- Chẩn đoán (trong top): {len(dx)} hàng → {out_dx.name}")
print(f"- Thủ thuật   (trong top): {len(pr)} hàng → {out_pr.name}")
print(f"- Xét nghiệm  (trong top): {len(lb)} hàng → {out_lab.name}")

# Nếu muốn xem nhanh:
print("\nVí dụ 3 dòng đầu mỗi bảng:")
print(dx.head(3))
print(pr.head(3))
print(lb.head(3))


Đã nạp vocab top: ICD=50, PROC=50, LAB=50

=== TÓM TẮT ===
- Chẩn đoán (trong top): 4 hàng → hadm_24151113_diagnoses_top.csv
- Thủ thuật   (trong top): 3 hàng → hadm_24151113_procedures_top.csv
- Xét nghiệm  (trong top): 49 hàng → hadm_24151113_labs_top.csv

Ví dụ 3 dòng đầu mỗi bảng:
   subject_id   hadm_id icd_code  icd_version  \
0    14471034  24151113     N179           10   
1    14471034  24151113     E871           10   
2    14471034  24151113      Z66           10   

                          long_title  
0  Acute kidney failure, unspecified  
1   Hypo-osmolality and hyponatremia  
2                 Do not resuscitate  
   subject_id   hadm_id icd_code  icd_version  \
0    14471034  24151113  5A1945Z           10   
1    14471034  24151113  0BH17EZ           10   
2    14471034  24151113  02HV33Z           10   

                                          long_title  
0   Respiratory Ventilation, 24-96 Consecutive Hours  
1  Insertion of Endotracheal Airway into Trachea,...  

In [ ]:
import pandas as pd
from pathlib import Path

# Đường dẫn chính (nhớ rằng notebook nằm trong /notebooks)
subset_dir = Path("../data/mimiciv_subset")

# Các file cần đọc
target_files = [
    "admissions.csv",
    "discharge.csv.gz",
    "icd.csv",
    "procedures.csv",
    "patients.csv",
    "labs.csv",
    "d_icd_diagnoses.csv",
    "d_icd_procedures.csv",
]

def explore_csv(path: Path, nrows=5):
    """In thông tin sơ bộ của 1 file CSV"""
    print(f"\n=== {path.name} ===")
    try:
        df = pd.read_csv(path, compression='infer', nrows=nrows)
        print(f"Shape (approx): {pd.read_csv(path, compression='infer', usecols=[0]).shape[0]} rows × {len(df.columns)} cols")
        print("Columns:", list(df.columns))
        print("\nSample rows:")
        display(df.head())
    except Exception as e:
        print("⚠️ Error reading file:", e)

# Duyệt từng file
for file in target_files:
    path = subset_dir / file
    if path.exists():
        explore_csv(path)
    else:
        print(f"❌ Missing file: {path}")


hadm_24151113_procedures_top.csv
.DS_Store
diagnoses_27783915.csv
admissions.csv
discharge_example.csv
discharge.csv.gz
hadm_24151113_diagnoses_top.csv
sample_discharge.csv
hadm_24151113_labs_top.csv
icd.csv
procedures.csv
summary_27783915.csv
discharge_full.csv.gz
labs_27783915.csv
procedures_27783915.csv
24151113_diagnoses.csv
patients.csv
24151113_labs.csv
labs.csv
d_icd_diagnoses.csv
24151113_procedures.csv
d_icd_procedures.csv


In [1]:
import pandas as pd
from pathlib import Path

# Đường dẫn chính (nhớ rằng notebook nằm trong /notebooks)
subset_dir = Path("../data/mimiciv_subset")

# Các file cần đọc
target_files = [
    "admissions.csv",
    "discharge.csv.gz",
    "icd.csv",
    "procedures.csv",
    "patients.csv",
    "labs.csv",
    "d_icd_diagnoses.csv",
    "d_icd_procedures.csv",
]

def explore_csv(path: Path, nrows=5):
    """In thông tin sơ bộ của 1 file CSV"""
    print(f"\n=== {path.name} ===")
    try:
        df = pd.read_csv(path, compression='infer', nrows=nrows)
        print(f"Shape (approx): {pd.read_csv(path, compression='infer', usecols=[0]).shape[0]} rows × {len(df.columns)} cols")
        print("Columns:", list(df.columns))
        print("\nSample rows:")
        display(df.head())
    except Exception as e:
        print("⚠️ Error reading file:", e)

# Duyệt từng file
for file in target_files:
    path = subset_dir / file
    if path.exists():
        explore_csv(path)
    else:
        print(f"❌ Missing file: {path}")



=== admissions.csv ===
Shape (approx): 56296 rows × 5 cols
Columns: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime']

Sample rows:


,subject_id,hadm_id,admittime,dischtime,deathtime
0,16904137,21081215,2105-10-04 17:26:00,2105-10-12 11:11:00,NaN
1,18106347,24305596,2110-01-11 10:14:00,2110-01-15 17:31:00,NaN
2,16284044,23864737,2110-01-11 19:58:00,2110-01-17 16:00:00,NaN
3,12257372,23964251,2110-01-13 07:15:00,2110-01-19 14:43:00,NaN
4,13201095,28453791,2110-01-18 14:46:00,2110-01-25 09:40:00,2110-01-25 09:40:00



=== discharge.csv.gz ===
Shape (approx): 87544 rows × 3 cols
Columns: ['hadm_id', 'charttime', 'text']

Sample rows:


,hadm_id,charttime,text
0,20000147,2121-09-03,Chief Complaint:\nchest pain\n\nMajor Surgical...
1,20000235,2139-12-03,Chief Complaint:\nAltered mental status\n\nMaj...
2,20000374,2132-12-07,Chief Complaint:\nabdominal pain\n\nMajor Surg...
3,20000471,2171-04-13,Chief Complaint:\nMalaise\n\nMajor Surgical or...
4,20000629,2161-12-26,Chief Complaint:\nmental status change\n\nMajo...



=== icd.csv ===
Shape (approx): 50 rows × 2 cols
Columns: ['icd_code', 'count']

Sample rows:


...

,itemid,count,label
0,50971,2648172,Potassium
1,50983,2625912,Sodium
2,50902,2601418,Chloride
3,50912,2587046,Creatinine
4,51221,2580696,Hematocrit



=== d_icd_diagnoses.csv ===
Shape (approx): 50 rows × 4 cols
Columns: ['icd_code', 'icd_version', 'long_title', 'count']

Sample rows:


,icd_code,icd_version,long_title,count
0,4019,9,Unspecified essential hypertension,102368
1,E785,10,"Hyperlipidemia, unspecified",84570
2,I10,10,Essential (primary) hypertension,83775
3,2724,9,Other and unspecified hyperlipidemia,67293
4,Z87891,10,Personal history of nicotine dependence,62806



=== d_icd_procedures.csv ===
Shape (approx): 50 rows × 5 cols
Columns: ['icd_code', 'icd_version', 'long_title', 'proc_code', 'count']

Sample rows:


,icd_code,icd_version,long_title,proc_code,count
0,3893,9,"Venous catheterization, not elsewhere classified",3893,14644
1,02HV33Z,10,Insertion of Infusion Device into Superior Ven...,02HV33Z,14353
2,8938,9,Other nonoperative respiratory measurements,8938,10519
3,3897,9,Central venous catheter placement with guidance,3897,10347
4,8856,9,Coronary arteriography using two catheters,8856,9549


In [2]:
schemas = {
    "admissions.csv": ["subject_id", "hadm_id", "admittime", "dischtime", "deathtime"],
    "discharge.csv.gz": ["hadm_id", "charttime", "text"],
    "patients.csv": ["subject_id", "gender", "anchor_age", "anchor_year"],
    "icd.csv": ["icd_code", "count"],
    "procedures.csv": ["proc_code", "count"],
    "labs.csv": ["itemid", "count", "label"],
    "d_icd_diagnoses.csv": ["icd_code", "icd_version", "long_title", "count"],
    "d_icd_procedures.csv": ["icd_code", "icd_version", "long_title", "proc_code", "count"],
}

for name, cols in schemas.items():
    print(f"{name}: {len(cols)} columns -> {cols}")


admissions.csv: 5 columns -> ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime']
discharge.csv.gz: 3 columns -> ['hadm_id', 'charttime', 'text']
patients.csv: 4 columns -> ['subject_id', 'gender', 'anchor_age', 'anchor_year']
icd.csv: 2 columns -> ['icd_code', 'count']
procedures.csv: 2 columns -> ['proc_code', 'count']
labs.csv: 3 columns -> ['itemid', 'count', 'label']
d_icd_diagnoses.csv: 4 columns -> ['icd_code', 'icd_version', 'long_title', 'count']
d_icd_procedures.csv: 5 columns -> ['icd_code', 'icd_version', 'long_title', 'proc_code', 'count']


In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

# --- Đường dẫn ---
DATA_ROOT = Path("..") / "data" / "proc"
PARQUET_PATH = DATA_ROOT / "examples.parquet"
VOCAB_PATH   = DATA_ROOT / "vocab_meta.json"

# --- 1️⃣ Đọc dữ liệu ---
df = pd.read_parquet(PARQUET_PATH)
print(f"✅ Đã đọc {len(df):,} lượt nhập viện từ {PARQUET_PATH.name}")

# --- 2️⃣ Kiểm tra cột có tồn tại không ---
expected_cols = ["hadm_id", "y_icd", "y_proc", "y_lab"]
missing_cols = [c for c in expected_cols if c not in df.columns]
if missing_cols:
    print("⚠️ Thiếu cột:", missing_cols)

# --- 3️⃣ Tính thống kê nhãn ---
def check_multihot(name):
    n_total = len(df)
    n_all_zero = (df[name].apply(lambda a: np.sum(a) == 0)).sum()
    mean_pos = df[name].apply(lambda a: np.sum(a)).mean()
    print(f"\n[{name}]")
    print(f"- Tỷ lệ vector toàn 0: {n_all_zero / n_total:.3%}")
    print(f"- Trung bình số nhãn dương / mẫu: {mean_pos:.3f}")

check_multihot("y_icd")
check_multihot("y_proc")
check_multihot("y_lab")

# --- 4️⃣ Kiểm tra dữ liệu rỗng hoặc text bị thiếu ---
missing_text = (df["text"].str.strip() == "").sum()
print(f"\n🧾 Số ca không có ghi chú text: {missing_text} / {len(df)}")

# --- 5️⃣ Tải vocab để đối chiếu kích thước ---
with open(VOCAB_PATH) as f:
    vocab_meta = json.load(f)
print("\n📘 Thông tin vocab:")
print(f"- ICD vocab size:  {vocab_meta.get('n_icd', '?')}")
print(f"- PROC vocab size: {vocab_meta.get('n_proc', '?')}")
print(f"- LAB vocab size:  {vocab_meta.get('n_lab', '?')}")

# --- 6️⃣ Kiểm tra độ dài vector có khớp với vocab không ---
for name, key in zip(["y_icd", "y_proc", "y_lab"],
                     ["n_icd", "n_proc", "n_lab"]):
    n_vocab = vocab_meta.get(key)
    if n_vocab:
        ok = df[name].apply(lambda a: len(a) == n_vocab).all()
        print(f"{name} khớp vocab ({n_vocab} nhãn):", "✅ OK" if ok else "❌ Sai độ dài")

# --- 7️⃣ Hiển thị ví dụ mẫu ---
print("\n🧩 Ví dụ một mẫu bất kỳ:")
sample = df.sample(1).iloc[0]
print("hadm_id:", sample["hadm_id"])
print("gender:", sample["gender"], "| age:", sample["age_at_admit"])
print("ICD > 0:", np.where(np.array(sample["y_icd"]) == 1)[0])
print("PROC > 0:", np.where(np.array(sample["y_proc"]) == 1)[0])
print("LAB  > 0:", np.where(np.array(sample["y_lab"]) == 1)[0])


✅ Đã đọc 56,296 lượt nhập viện từ examples.parquet

[y_icd]
- Tỷ lệ vector toàn 0: 0.000%
- Trung bình số nhãn dương / mẫu: 1.979

[y_proc]
- Tỷ lệ vector toàn 0: 0.000%
- Trung bình số nhãn dương / mẫu: 1.979

[y_lab]
- Tỷ lệ vector toàn 0: 0.000%
- Trung bình số nhãn dương / mẫu: 2.445

🧾 Số ca không có ghi chú text: 0 / 56296

📘 Thông tin vocab:
- ICD vocab size:  50
- PROC vocab size: 50
- LAB vocab size:  50
y_icd khớp vocab (50 nhãn): ✅ OK
y_proc khớp vocab (50 nhãn): ✅ OK
y_lab khớp vocab (50 nhãn): ✅ OK

🧩 Ví dụ một mẫu bất kỳ:
hadm_id: 28611522
gender: M | age: 53
ICD > 0: [25 30 41]
PROC > 0: [25 30 41]
LAB  > 0: [21]
